This notebook was run on  kaggle to generate the embeddings for hotel facilities

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load a pre-trained model and move it to GPU
model_name = "roberta-base"  # or use 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# Function to generate embeddings on GPU
def get_embedding(text):
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)  # Move tokens to GPU
    with torch.no_grad():
        output = model(**tokens)
    embedding = output.last_hidden_state[:, 0, :].cpu().numpy()  # Move back to CPU for further processing
    return embedding.squeeze()

# Generate embeddings for all hotels
import pandas as pd

# Load your dataset
df = pd.read_csv("/kaggle/input/indian-hotels/indian_hotels2.csv")
df = df[df['clean_text'].notnull() & (df['clean_text'].str.strip() != "")]

# Apply the embedding function to the cleaned_text column
df['embeddings'] = df['clean_text'].apply(lambda x: get_embedding(x))

import numpy as np

# Save embeddings as a NumPy file
np.save("/kaggle/working/hotel_embeddings.npy", df['embeddings'].to_list())

# Save the rest of the DataFrame (without embeddings) as a CSV
df.drop(columns=['embeddings']).to_csv("/kaggle/working/hotel_data.csv", index=False)


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
